In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gensim
import spacy
import nltk
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from gensim.models import Word2Vec
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
import tensorflow as tf
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Reproducibility/Sample Model/HateXplain2.csv")

In [ ]:
df = df[['post_tokens',	'Target_cat',	'Label_cat',	'final_rationales']]
df['post_tokens'] = df['post_tokens'].apply(lambda x: eval(x))
df.head()

,post_tokens,Target_cat,Label_cat,final_rationales
0,"[i, dont, think, im, getting, my, baby, them, ...",None,normal,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[we, cannot, continue, calling, ourselves, fem...",None,normal,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[nawt, yall, niggers, ignoring, me]",African,normal,"[0, 0, 0, 0, 0]"
3,"[<user>, i, am, bit, confused, coz, chinese, p...",Asian,hatespeech,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[this, bitch, in, whataburger, eating, a, burg...",Women,hatespeech,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
sentences = [tweet for tweet in df['post_tokens']]

In [ ]:
model_WV = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=5, workers=4, sg=0)

# Save Word2Vec model
model_WV.save('/content/drive/MyDrive/Reproducibility/Test_model/word2vec_model')

In [ ]:
word_vectors = model_WV.wv

In [ ]:
num_words = len(word_vectors.key_to_index.keys())
embedding_dim = 100
embedding_matrix = np.zeros((num_words, embedding_dim))
for i, word in enumerate(word_vectors.index_to_key):
    embedding_vector = word_vectors[word]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
target_encoder = LabelEncoder()
target_labels = target_encoder.fit_transform(df['Target_cat'])
y1 = tf.keras.utils.to_categorical(target_labels, num_classes=21)
Label_encoder = LabelEncoder()
Label_labels = Label_encoder.fit_transform(df['Label_cat'])
y2 = tf.keras.utils.to_categorical(Label_labels, num_classes=3)

In [ ]:
sequences = []
for tweet in df['post_tokens']:
    sequence = []
    for word in tweet:
        if word in model_WV.wv:
            sequence.append(model_WV.wv[word])
    sequences.append(sequence)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y1_train, y1_test, y2_train, y2_test = train_test_split(sequences, y1, y2, test_size=0.2, random_state=42)

In [ ]:
# Pad sequences to a fixed length
max_sequence_length = 100
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_sequence_length, padding='post', truncating='post')
padded_sequences_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_sequence_length, padding='post', truncating='post')

In [ ]:
x_train = padded_sequences
x_test = padded_sequences_test

In [ ]:
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, GlobalMaxPooling1D, Concatenate, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.losses import categorical_crossentropy

def build_model(max_sequence_length, embedding_dim, num_classes_community, num_classes_label):
    
    input_layer = Input(shape=(max_sequence_length, embedding_dim))
    conv_layer1 = Conv1D(filters=32, kernel_size=3, activation='relu', padding='same')(input_layer)
    batch_norm1 = BatchNormalization()(conv_layer1)
    conv_layer2 = Conv1D(filters=32, kernel_size=3, activation='relu', padding='same')(batch_norm1)
    batch_norm2 = BatchNormalization()(conv_layer2)

    skip_layer1 = Concatenate()([batch_norm1, batch_norm2])
    conv_layer3 = Conv1D(filters=32, kernel_size=3, activation='relu', padding='same')(skip_layer1)
    batch_norm3 = BatchNormalization()(conv_layer3)

    skip_layer2 = Concatenate()([batch_norm2, batch_norm3])
    conv_layer4 = Conv1D(filters=32, kernel_size=3, activation='relu', padding='same')(skip_layer2)
    batch_norm4 = BatchNormalization()(conv_layer4)

    skip_layer3 = Concatenate()([batch_norm3, batch_norm3])
    conv_layer5 = Conv1D(filters=32, kernel_size=3, activation='relu', padding='same')(skip_layer3)
    batch_norm5 = BatchNormalization()(conv_layer5)

    skip_layer4 = Concatenate()([batch_norm4, batch_norm5])
    conv_layer6 = Conv1D(filters=32, kernel_size=3, activation='relu', padding='same')(skip_layer4)
    batch_norm6 = BatchNormalization()(conv_layer6)

    drop1 = Dropout(0.5)(batch_norm6)
    pool_layer1 = MaxPooling1D(pool_size=5)(drop1)


    flatten_layer = Flatten()(pool_layer1)

    dense_layer1 = Dense(64, activation='relu')(flatten_layer)
    batch_norm7 = BatchNormalization()(dense_layer1)
    drop2 = Dropout(0.5)(batch_norm7)
    dense_layer2 = Dense(64, activation='relu')(drop2)
    batch_norm8 = BatchNormalization()(dense_layer2)
    drop3 = Dropout(0.5)(batch_norm8)
    dense_layer3 = Dense(32, activation='relu')(drop2)
    batch_norm9 = BatchNormalization()(dense_layer3)
    drop4 = Dropout(0.5)(batch_norm9)
  
    dense_layer4 = Dense(64, activation='relu')(flatten_layer)
    batch_norm10 = BatchNormalization()(dense_layer4)
    drop5 = Dropout(0.5)(batch_norm10)
    dense_layer5 = Dense(64, activation='relu')(drop5)
    batch_norm11 = BatchNormalization()(dense_layer5)
    drop6 = Dropout(0.5)(batch_norm11)
    dense_layer6 = Dense(32, activation='relu')(drop5)
    batch_norm12 = BatchNormalization()(dense_layer6)
    drop7 = Dropout(0.5)(batch_norm12)

    # Define the output layers
    output_community = Dense(num_classes_community, activation='softmax', name='community_output')(drop4)
    output_label = Dense(num_classes_label, activation='softmax', name='label_output')(drop7)

    # Compile the model with the custom loss function
    model = Model(inputs=input_layer, outputs=[output_community, output_label])
    model.compile(loss={'community_output': categorical_crossentropy,
                        'label_output': categorical_crossentropy},
                  optimizer='adam',
                  metrics={'community_output': 'accuracy',
                           'label_output': 'accuracy'})
    return model


In [ ]:
x_train.shape

(16118, 100, 100)

In [ ]:
model = build_model(max_sequence_length, embedding_dim, num_classes_community = 21, num_classes_label = 3 )

In [ ]:
history = model.fit(x= x_train, y = {'community_output': y1_train, 'label_output': y2_train}, validation_data = (x_test, {'community_output': y1_test, 'label_output': y2_test}), batch_size = 50, epochs = 50)

Epoch 1/50
323/323 [==============================] - 17s 23ms/step - loss: 4.6409 - community_output_loss: 3.1594 - label_output_loss: 1.4815 - community_output_accuracy: 0.1818 - label_output_accuracy: 0.3447 - val_loss: 3.4734 - val_community_output_loss: 2.3665 - val_label_output_loss: 1.1069 - val_community_output_accuracy: 0.3375 - val_label_output_accuracy: 0.3400
Epoch 2/50
323/323 [==============================] - 5s 16ms/step - loss: 3.5222 - community_output_loss: 2.3856 - label_output_loss: 1.1367 - community_output_accuracy: 0.3114 - label_output_accuracy: 0.3777 - val_loss: 3.1979 - val_community_output_loss: 2.1307 - val_label_output_loss: 1.0672 - val_community_output_accuracy: 0.3417 - val_label_output_accuracy: 0.4045
Epoch 3/50
323/323 [==============================] - 6s 20ms/step - loss: 3.2137 - community_output_loss: 2.1417 - label_output_loss: 1.0720 - community_output_accuracy: 0.3438 - label_output_accuracy: 0.3917 - val_loss: 3.1290 - val_community_output_l